In [73]:
team_id = '40'
split = "test_1"

In [74]:
from src.dataloader import base_loader

df, df_test = base_loader(split)

In [75]:
from src.embedding import base_embedder
from sklearn import preprocessing

X = base_embedder(df.words_str.values)
y_reg = df.score_compound.values
y_clf = preprocessing.LabelEncoder().fit_transform(df.sentiment.values)

X_test = base_embedder(df_test.words_str.values)

### Regression

In [76]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y_reg, test_size=0.2, random_state=42, stratify=df.sentiment)

# X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)

X_resampled = X_train
y_resampled = y_train

datasets = [
    [X_resampled, y_resampled],
    [X_val, y_val]
]

In [77]:
from sklearn.neural_network import MLPRegressor

reg = MLPRegressor((512, 256), learning_rate="adaptive").fit(X_train, y_train)

In [78]:
from src.evaluation import eval_reg

eval_reg(reg, datasets)


Split: train
	RMSE: 0.04
	MAE: 0.03

Split: valididation
	RMSE: 0.35
	MAE: 0.27


In [79]:
from src.utils import save_sub

save_sub(reg, X_test, split, 'reg')

### Classification

In [80]:
X_train, X_val, y_train, y_val = train_test_split(X, y_clf, test_size=0.2, random_state=42, stratify=y_clf)

X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)

datasets = [
    [X_resampled, y_resampled],
    [X_val, y_val]
]

In [81]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier((512, 256, 128, 64, 32), learning_rate="adaptive", max_iter=1000).fit(X_train, y_train)

In [82]:
from src.evaluation import eval_clf

eval_clf(clf, datasets)


Split: train
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4232
           1       0.98      1.00      0.99      4232
           2       1.00      0.98      0.99      4232

    accuracy                           0.99     12696
   macro avg       0.99      0.99      0.99     12696
weighted avg       0.99      0.99      0.99     12696


Split: validation
              precision    recall  f1-score   support

           0       0.43      0.55      0.48        96
           1       0.81      0.81      0.81      1058
           2       0.65      0.61      0.63       446

    accuracy                           0.74      1600
   macro avg       0.63      0.66      0.64      1600
weighted avg       0.74      0.74      0.74      1600



In [83]:
from src.utils import save_sub

save_sub(reg, X_test, split, 'clf')